# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Competiton: [link](https://www.kaggle.com/t/49ea0c385a974db5919ec67299ba2e6b)
  - Slide: [link](https://docs.google.com/presentation/d/1LDAW0GGrC9B6D7dlNdYzQL6D60-iKgFr/edit?usp=sharing&ouid=104280564485377739218&rtpof=true&sd=true)
  - Data: [link](https://github.com/googly-mingto/ML2023HW4/releases)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

In [1]:
# !wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partaa
# !wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partab
# !wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partac
# !wget https://github.com/googly-mingto/ML2023HW4/releases/download/data/Dataset.tar.gz.partad

# !cat Dataset.tar.gz.part* > Dataset.tar.gz
# !rm Dataset.tar.gz.partaa
# !rm Dataset.tar.gz.partab
# !rm Dataset.tar.gz.partac
# !rm Dataset.tar.gz.partad
# # unzip the file
# !tar zxf Dataset.tar.gz
# !rm Dataset.tar.gz

In [2]:
# !tar zxf Dataset.tar.gz

In [3]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [4]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
			return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [5]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
	def __init__(self, d_model=256, n_spks=600, dropout=0.25):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=256, nhead=16
		)
		# self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.Sigmoid(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# out: (batch size, length, d_model)
		out = self.prenet(mels)
		# out: (length, batch size, d_model)
		out = out.permute(1, 0, 2)
		# The encoder layer expect features in the shape of (length, batch size, d_model).
		out = self.encoder_layer(out)
		# out: (batch size, length, d_model)
		out = out.transpose(0, 1)
		# mean pooling
		stats = out.mean(dim=1)

		# out: (batch, n_spks)
		out = self.pred_layer(stats)
		return out

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [7]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [8]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [9]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

# Main function

In [ ]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"save_path": "model.ckpt",
		"batch_size": 128,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 10000000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
# 	model.load_state_dict(torch.load(save_path)) # continue
    
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-2)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")
			with open("./log.txt",'a') as f:
				f.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})\n")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [00:21<00:00, 94.59 step/s, accuracy=0.02, loss=5.42, step=2000] 
Valid:  99% 5632/5667 [00:00<00:00, 8095.75 uttr/s, accuracy=0.03, loss=5.41] 
Train: 100% 2000/2000 [00:20<00:00, 96.85 step/s, accuracy=0.16, loss=4.56, step=4000] 
Valid:  99% 5632/5667 [00:00<00:00, 12997.04 uttr/s, accuracy=0.10, loss=4.67]
Train: 100% 2000/2000 [00:20<00:00, 96.32 step/s, accuracy=0.19, loss=3.98, step=6000] 
Valid:  99% 5632/5667 [00:00<00:00, 13199.23 uttr/s, accuracy=0.17, loss=4.17]
Train: 100% 2000/2000 [00:20<00:00, 95.72 step/s, accuracy=0.22, loss=3.85, step=8000] 
Valid:  99% 5632/5667 [00:00<00:00, 13127.42 uttr/s, accuracy=0.23, loss=3.81]
Train: 100% 2000/2000 [00:20<00:00, 96.06 step/s, accuracy=0.28, loss=3.47, step=1e+4] 
Valid:  99% 5632/5667 [00:00<00:00, 13109.18 uttr/s, accuracy=0.26, loss=3.51]
Train:   1% 21/2000 [00:00<00:19, 99.84 step/s, accuracy=0.30, loss=3.51, step=1e+4]

Step 10000, best model saved. (accuracy=0.2635)


Train: 100% 2000/2000 [00:20<00:00, 95.96 step/s, accuracy=0.35, loss=3.46, step=12000] 
Valid:  99% 5632/5667 [00:00<00:00, 13435.35 uttr/s, accuracy=0.31, loss=3.29]
Train: 100% 2000/2000 [00:20<00:00, 95.96 step/s, accuracy=0.37, loss=3.16, step=14000] 
Valid:  99% 5632/5667 [00:00<00:00, 13111.38 uttr/s, accuracy=0.35, loss=3.09]
Train: 100% 2000/2000 [00:20<00:00, 95.65 step/s, accuracy=0.40, loss=2.93, step=16000] 
Valid:  99% 5632/5667 [00:00<00:00, 13056.46 uttr/s, accuracy=0.38, loss=2.94]
Train: 100% 2000/2000 [00:20<00:00, 95.66 step/s, accuracy=0.46, loss=2.71, step=18000] 
Valid:  99% 5632/5667 [00:00<00:00, 13230.92 uttr/s, accuracy=0.41, loss=2.78]
Train: 100% 2000/2000 [00:20<00:00, 95.85 step/s, accuracy=0.55, loss=2.21, step=2e+4]  
Valid:  99% 5632/5667 [00:00<00:00, 12940.57 uttr/s, accuracy=0.42, loss=2.67]
Train:   1% 20/2000 [00:00<00:19, 102.61 step/s, accuracy=0.47, loss=2.60, step=2e+4]

Step 20000, best model saved. (accuracy=0.4235)


...

Train: 100% 2000/2000 [00:40<00:00, 49.78 step/s, accuracy=1.00, loss=0.01, step=9.2e+6]
Valid:  99% 5632/5667 [00:00<00:00, 10040.59 uttr/s, accuracy=0.84, loss=0.79]
Train: 100% 2000/2000 [00:40<00:00, 49.93 step/s, accuracy=0.99, loss=0.01, step=9.2e+6]
Valid:  99% 5632/5667 [00:00<00:00, 8315.23 uttr/s, accuracy=0.83, loss=0.81] 
Train: 100% 2000/2000 [00:40<00:00, 49.81 step/s, accuracy=1.00, loss=0.02, step=9206000]
Valid:  99% 5632/5667 [00:00<00:00, 10245.85 uttr/s, accuracy=0.83, loss=0.80]
Train: 100% 2000/2000 [00:39<00:00, 50.07 step/s, accuracy=1.00, loss=0.01, step=9208000]
Valid:  99% 5632/5667 [00:00<00:00, 9789.15 uttr/s, accuracy=0.84, loss=0.81] 
Train: 100% 2000/2000 [00:34<00:00, 58.32 step/s, accuracy=1.00, loss=0.01, step=9210000]
Valid:  99% 5632/5667 [00:00<00:00, 9378.28 uttr/s, accuracy=0.84, loss=0.79] 
Train:   1% 19/2000 [00:00<00:21, 93.10 step/s, accuracy=0.99, loss=0.02, step=9210019]

Step 9210000, best model saved. (accuracy=0.8475)


Train: 100% 2000/2000 [00:23<00:00, 83.84 step/s, accuracy=1.00, loss=0.02, step=9212000]
Valid:  99% 5632/5667 [00:00<00:00, 9091.26 uttr/s, accuracy=0.83, loss=0.82] 
Train: 100% 2000/2000 [00:24<00:00, 81.65 step/s, accuracy=1.00, loss=0.02, step=9214000]
Valid:  99% 5632/5667 [00:00<00:00, 9424.37 uttr/s, accuracy=0.84, loss=0.81] 
Train: 100% 2000/2000 [00:25<00:00, 78.44 step/s, accuracy=0.99, loss=0.02, step=9216000]
Valid:  99% 5632/5667 [00:00<00:00, 10748.24 uttr/s, accuracy=0.84, loss=0.83]
Train: 100% 2000/2000 [00:37<00:00, 52.83 step/s, accuracy=1.00, loss=0.01, step=9218000]
Valid:  99% 5632/5667 [00:00<00:00, 9046.45 uttr/s, accuracy=0.83, loss=0.81] 
Train: 100% 2000/2000 [00:39<00:00, 50.07 step/s, accuracy=1.00, loss=0.01, step=9220000]
Valid:  99% 5632/5667 [00:00<00:00, 10133.29 uttr/s, accuracy=0.84, loss=0.79]
Train:   0% 9/2000 [00:00<00:41, 48.31 step/s, accuracy=1.00, loss=0.01, step=9220009]

Step 9220000, best model saved. (accuracy=0.8475)


Train: 100% 2000/2000 [00:40<00:00, 49.98 step/s, accuracy=0.99, loss=0.02, step=9222000]
Valid:  99% 5632/5667 [00:00<00:00, 8833.53 uttr/s, accuracy=0.85, loss=0.77] 
Train: 100% 2000/2000 [00:39<00:00, 50.17 step/s, accuracy=1.00, loss=0.01, step=9224000]
Valid:  99% 5632/5667 [00:00<00:00, 9381.25 uttr/s, accuracy=0.83, loss=0.83] 
Train:  15% 294/2000 [00:06<00:33, 51.52 step/s, accuracy=1.00, loss=0.02, step=9224294]

# Inference

## Dataset of inference

In [ ]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [ ]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	model2 = Classifier(n_spks=speaker_num).to(device)
	model2.load_state_dict(torch.load('./94925.ckpt'))
	model2.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			outs += model2(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())